In [ ]:
# -*- coding: utf-8 -*-
"""bayesian_final_project.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1hyQoKJ20GiNwqjGBk53IRu95Hd3XvqZO

My implementations
"""

import numpy as np
from numba import njit,prange
from functools import partial

def multinomial_resampling(particles, weights):
    """
    Resample particles based on their weights using multinomial resampling.
    :param particles: An array of particles with shape (num_particles, particle_dimension).
    :param weights: An array of weights with shape (num_particles,).
    :return: The resampled particles.
    """
    num_particles = particles.shape[0]

    # Normalize the weights
    # normalized_weights = weights / np.sum(weights)

    # Resample particles using the multinomial distribution
    resampled_indices = np.random.choice(num_particles, size=num_particles, p=weights)

    # Select the resampled particles
    resampled_particles = particles[resampled_indices]

    return resampled_particles

@njit(parallel=True)
def multinomial_resampling_numba(weights):
    N = len(weights)
    indices = np.zeros(N, dtype=np.int32)
    cumsum = np.cumsum(weights)
    cumsum /= cumsum[-1]
    random_nums = np.random.rand(N)
    for i in prange(N):
        indices[i] = np.searchsorted(cumsum, random_nums[i])
    return indices

# @njit(parallel=True)
# def multinomial_resampling_numba(particles, weights):
#     """Perform multinomial resampling on the particles."""
#     num_particles = particles.shape[0]
#     indices = np.zeros(num_particles, dtype=np.int64)
#     cumsum_weights = np.cumsum(weights)
#     cumsum_weights[-1] = 1.  # Ensure cumsum_weights[-1] = 1.0
#     for i in range(num_particles):
#         r = np.random.rand()
#         j = 0
#         while r > cumsum_weights[j]:
#             j += 1
#         indices[i] = j
#     return particles[indices]

import jax.numpy as jnp
from jax import random, jit
import jax

def multinomial_resampling_jax(particles, weights, rng_key):
    cumsum_weights = jnp.cumsum(weights)
    cumsum_weights /= cumsum_weights[-1]

    # Resample particles
    u = random.uniform(rng_key, (particles.shape[0],))
    indices = jnp.searchsorted(cumsum_weights, u)
    # resampled_particles = particles[indices]

    # Assign equal weights to each particle
    # resampled_weights = jnp.ones(weights.shape) / weights.shape[0]

    return indices

@jit
def multinomial_resampling_jit(particles, weights, rng_key):
    cumsum_weights = jnp.cumsum(weights)
    cumsum_weights /= cumsum_weights[-1]

    # Resample particles
    u = random.uniform(rng_key, (particles.shape[0],))
    indices = jnp.searchsorted(cumsum_weights, u)
    resampled_particles = particles[indices]

    # Assign equal weights to each particle
    # resampled_weights = jnp.ones(weights.shape) / weights.shape[0]

    return resampled_particles

# @jit
# def searchsorted_jit(cumsum, x):
#     # Find the indices where x would be inserted into cumsum
#     # Assumes cumsum is sorted in ascending order
#     return jnp.searchsorted(cumsum, x)

# @jit
# def multinomial_resampling_jit(particles, weights, rng):
#     num_particles = particles.shape[0]

#     # Calculate the cumulative sum of the weights
#     cumsum = jnp.cumsum(weights)

#     # # Normalize the weights
#     # weights_norm = weights / jnp.sum(weights)

#     # Resample the particles
#     idxs = []
#     for i in range(num_particles):
#         # Draw a random number in [0,resample_particles_multinomial 1/num_particles)
#         u = random.uniform(rng, shape=(1,)) / num_particles

#         # Find the index of the first element in cumsum greater than u
#         idx = searchsorted_jit(cumsum, u)

#         # Append the index to the list
#         idxs.append(idx)

#         # Update the cumulative sum to account for the weight of the selected particle
#         # cumsum = jax.ops.index_update(cumsum, idx, cumsum[idx] - weights[idx])
#         cumsum = cumsum.at[idx].set(cumsum[idx] - weights[idx])

#     # Convert the indices to an array
#     idxs = jnp.array(idxs)

#     # Use advanced indexing to select the resampled particles
#     resampled_particles = particles[idxs]

#     return resampled_particles

# @partial(jit, static_argnums=(2,3))
# def multinomial_resampling_jit(particles, weights, rng_key):
#     cumsum_weights = jnp.cumsum(weights)
#     cumsum_weights /= cumsum_weights[-1]

#     # Resample particles
#     u = random.uniform(rng_key, (particles.shape[0],))
#     indices = jnp.searchsorted(cumsum_weights, u)
#     # resampled_particles = particles[indices]

#     # Assign equal weights to each particle
#     # resampled_weights = jnp.ones(weights.shape) / weights.shape[0]

#     return indices

# def multinomial_resampling_jax(particles, weights, key):
#     num_particles, particle_dimension = particles.shape

#     # Normalize the weights
#     # weights = weights / jnp.sum(weights)

#     # Resample particles using the multinomial distribution
#     key, subkey = random.split(key)
#     resampled_indices = random.categorical(subkey, weights, shape=(num_particles,))
#     resampled_particles = particles[resampled_indices]

#     return resampled_particles


# def multinomial_resampling_jax(particles, weights,rng_key):
#     num_particles = particles.shape[0]

#     # Compute cumulative sum of weights
#     cumsum = jnp.cumsum(weights)

#     # Sample indices using binary search
#     indices = jax.lax.map(lambda key: searchsorted(cumsum, key), jax.random.split(rng_key, num=num_particles))

#     # Resample particles based on indices
#     resampled_particles = particles[indices]

#     # Normalize weights
#     # resampled_weights = jnp.full(num_particles, 1.0 / num_particles)

#     return resampled_particles


# def searchsorted(cumsum, key):
#     lo = 0
#     hi = cumsum.shape[0] - 1
#     while lo < hi:
#         mid = (lo + hi) // 2
#         if cumsum[mid] < key:
#             lo = mid + 1
#         else:
#             hi = mid
#     return lo

# def multinomial_resampling_jax(particresample_particles_multinomialles, weights,key):
#     M = particles.shape[0]
#     cum_weights = jnp.zeros(M+1)
#     cumsum_fun = lambda c, w: (c + w, c + w)
#     cum_weights = jax.lax.scan(cumsum_fun, cum_weights[:-1], weights)
#     cum_weights = jnp.concatenate((jnp.array([0.0]), cum_weights[0]), axis=0)
#     cum_weights = cum_weights[1:]
#     cum_weights /= cum_weights[-1]
#     uniform_samples = jax.random.uniform(key, (M,), dtype=jnp.float32)
#     inds = jnp.searchsorted(cum_weights, uniform_samples)
#     return particles[inds], jnp.ones(M) / M

# @jit
# def multinomial_resampling_jit(particles, weights,key):
#     M = particles.shape[0]
#     cum_weights = jnp.zeros(M+1)
#     cumsum_fun = lambda c, w: (c + w, c + w)
#     cum_weights = jax.lax.scan(cumsum_fun, cum_weights[:-1], weights)
#     cum_weights = jnp.concatenate((jnp.array([0.0]), cum_weights[0]), axis=0)
#     cum_weights = cum_weights[1:]
#     cum_weights /= cum_weights[-1]
#     uniform_samples = jax.random.uniform(key, (M,), dtype=jnp.float32)
#     inds = jnp.searchsorted(cum_weights, uniform_samples)
#     return particles[inds], jnp.ones(M) / M

# @jit
# def multinomial_resampling_jit(particles, weights,rng_key):
#     num_particles = particles.shape[0]

#     # Compute cumulative sum of weights
#     cumsum = jnp.cumsum(weights)

#     # Sample indices using binary search
#     indices = jax.lax.map(lambda key: searchsorted_jit(cumsum, key), jax.random.split(rng_key, num=num_particles))

#     # Resample particles based on indices
#     resampled_particles = particles[indices]

#     # Normalize weights
#     # resampled_weights = jnp.full(num_particles, 1.0 / num_particles)

#     return resampled_particles

# @jit
# def searchsorted_jit(cumsum, key):
#     lo = 0
#     hi = cumsum.shape[0] - 1
#     while lo < hi:
#         mid = (lo + hi) // 2
#         if cumsum[mid] < key:
#             lo = mid + 1
#         else:
#             hi = mid
#     return lo

# @jit
# def multinomial_resampling_jit(particles, weights, key):
#     num_particles, particle_dimension = particles.shape

#     # Normalize the weights
#     # weights = weights / np.sum(weights)

#     # Resample particles using the multinomial distribution
#     key, subkey = random.split(key)
#     resampled_indices = random.categorical(subkey, weights, shape=(num_particles,))
#     resampled_particles = particles[resampled_indices]

#     return resampled_particles

def systematic_resampling(particles, weights):
    num_particles = particles.shape[0]
    step_size = 1.0 / num_particles
    r = np.random.uniform(0, step_size)
    cumulative_weights = np.cumsum(weights)

    indices = np.zeros(num_particles, dtype=int)
    i = 0
    for m in range(num_particles):
        while r > cumulative_weights[i]:
            i += 1
        indices[m] = i
        r += step_size
        if r >= 1.0:
            r -= 1.0
            i = 0

    return particles[indices]

@njit(parallel=True)
def systematic_resampling_numba(particles, weights):
    """Perform systematic resampling on the particles."""
    num_particles = particles.shape[0]
    indices = np.zeros(num_particles, dtype=np.int64)
    cumsum_weights = np.cumsum(weights)
    cumsum_weights[-1] = 1.  # Ensure cumsum_weights[-1] = 1.0
    step = 1.0 / num_particles
    u = np.random.rand() * step
    i = 0
    for j in range(num_particles):
        while cumsum_weights[i] < u:
            i += 1
        indices[j] = i
        u += step
    return particles[indices]

def systematic_resampling_jax(weights, key):
    n = len(weights)
    indices = jnp.arange(n)
    cum_weights = jnp.cumsum(weights)
    step = cum_weights[-1] / n
    u = (jnp.arange(n) + jax.random.uniform(key, (n,))) * step
    j = jnp.zeros((), dtype=jnp.int32)
    def body_fn(i, j):
        j = jax.lax.cond(cum_weights[j] < u[i],
                         lambda _: j + 1,
                         lambda _: j,
                         operand=None)
        return j, ()
    _, j = jax.lax.scan(body_fn, j, jnp.arange(n))
    # indices = jax.ops.index_update(indices, jnp.arange(n), indices[j])
    indices = indices.at[jnp.arange(n)].set(indices[j])
    return indices

@jit
def systematic_resampling_jit(weights, key):
    n = len(weights)
    indices = jnp.arange(n)
    cum_weights = jnp.cumsum(weights)
    step = cum_weights[-1] / n
    u = (jnp.arange(n) + jax.random.uniform(key, (n,))) * step
    j = jnp.zeros((), dtype=jnp.int32)
    def body_fn(i, j):
        j = jax.lax.cond(cum_weights[j] < u[i],
                         lambda _: j + 1,
                         lambda _: j,
                         operand=None)
        return j, ()
    _, j = jax.lax.scan(body_fn, j, jnp.arange(n))
    # indices = jax.ops.index_update(indices, jnp.arange(n), indices[j])
    indices = indices.at[jnp.arange(n)].set(indices[j])
    return indices

# num_particles = 100000
particle_dimension = 1
# particles = np.random.randn(num_particles, particle_dimension)
# weights = np.random.rand(num_particles)
seed = 42
key = random.PRNGKey(seed)
import time

particle_lst = [1,10,100,1000,10000,100000,1000000,10000000]
# particle_lst = [1,10,1000,10000]
multi_np_times = []
multi_numba_times = []
multi_jax_times = []
multi_jax_jit_times = []

sys_np_times = []
sys_numba_times = []
sys_jax_times = []
sys_jax_jit_times = []

num_iters = 10
for n_part in particle_lst:
    print("Number of particles:", n_part)
    particles = np.random.randn(n_part, particle_dimension)
    weights = np.ones(n_part)/n_part
    particles_jax = jnp.array(particles)
    weights_jax = jnp.array(weights)

    elapsed_time = 0
    print("multi numpy")
    for i in range(num_iters):
        start_time = time.time()
        stuff = multinomial_resampling(particles,weights)
        elapsed_time += time.time()-start_time
    multi_np_times.append(elapsed_time/num_iters)
    del stuff

    elapsed_time=0
    print("multi numba")
    stuff = multinomial_resampling_numba(weights)
    for i in range(num_iters):
        start_time = time.time()
        stuff = multinomial_resampling_numba(weights)
        particles[stuff]
        elapsed_time += time.time()-start_time
    multi_numba_times.append(elapsed_time/num_iters)
    del stuff

    print("multi jax")
    elapsed_time=0
    for i in range(num_iters):
        start_time = time.time()
        stuff = (multinomial_resampling_jax(particles_jax,weights_jax,key)).block_until_ready()
        elapsed_time += time.time()-start_time
    multi_jax_times.append(elapsed_time/num_iters)
    del stuff

    print("multi jax jit")
    elapsed_time = 0
    multinomial_resampling_jit(particles_jax,weights_jax,key).block_until_ready()
    for i in range(num_iters):
        start_time = time.time()
        stuff = multinomial_resampling_jit(particles_jax,weights_jax,key).block_until_ready()
        elapsed_time += time.time()-start_time
    multi_jax_jit_times.append(elapsed_time/num_iters)
    del stuff

    print("sys numpy")
    elapsed_time = 0
    for i in range(num_iters):
        start_time = time.time()
        stuff = systematic_resampling(particles,weights)
        elapsed_time += time.time()-start_time
    sys_np_times.append(elapsed_time/num_iters)
    del stuff

    print("sys numba")
    elapsed_time = 0
    stuff = systematic_resampling_numba(particles,weights)
    for i in range(num_iters):
        start_time = time.time()
        stuff = systematic_resampling_numba(particles,weights)
        elapsed_time += time.time()-start_time
    sys_numba_times.append(elapsed_time/num_iters)
    del stuff

    print("sys jax")
    elapsed_time = 0
    for i in range(num_iters):
        start_time = time.time()
        stuff = systematic_resampling_jax(weights_jax,key).block_until_ready()
        particles_jax[stuff]
        elapsed_time += time.time()-start_time
    sys_jax_times.append(elapsed_time/num_iters)
    del stuff

    print("sys jax jit")
    elapsed_time = 0
    systematic_resampling_jit(weights_jax,key).block_until_ready()
    for i in range(num_iters):
        start_time = time.time()
        stuff = systematic_resampling_jit(weights_jax,key).block_until_ready()
        particles_jax[stuff]
        elapsed_time += time.time()-start_time
    sys_jax_jit_times.append(elapsed_time/num_iters)
    del stuff

    del particles 
    del weights 
    del particles_jax
    del weights_jax

"""Create the plots"""

print(multi_np_times)
print(multi_numba_times)
print(multi_jax_times)
print(multi_jax_jit_times)

import matplotlib.pyplot as plt

multi_cpp = np.genfromtxt("execution_times_multinomial.csv",delimiter=',')
sys_cpp = np.genfromtxt("execution_times_systematic.csv",delimiter=",")
print(multi_cpp[:,1])

fig, ax = plt.subplots(2,1,figsize=(4,12))
# plt.figure()
ax[0].plot(particle_lst,multi_np_times,label="numpy")
ax[0].plot(particle_lst,multi_numba_times,label="numba")
ax[0].plot(particle_lst,multi_jax_times,label="JAX")
ax[0].plot(particle_lst,multi_jax_jit_times,label="JAX JIT")
ax[0].plot(multi_cpp[:,0],multi_cpp[:,1],label="C++")
ax[0].semilogy()
ax[0].semilogx()
ax[0].set_xlabel("Number of particles")
ax[0].set_ylabel("Execution time (s)")
ax[0].legend()
ax[0].set_title("Multinomial Resampling")
# plt.tight_layout()

# plt.figure()
ax[1].plot(particle_lst,sys_np_times,label="numpy")
ax[1].plot(particle_lst,sys_numba_times,label="numba")
ax[1].plot(particle_lst,sys_jax_times,label="JAX")
ax[1].plot(particle_lst,sys_jax_jit_times,label="JAX JIT")
ax[1].plot(sys_cpp[:,0],sys_cpp[:,1],label="C++")
ax[1].semilogy()
ax[1].semilogx()
ax[1].set_xlabel("Number of particles")
ax[1].set_ylabel("Execution time (s)")
ax[1].legend()
ax[1].set_title("Systematic Resampling")
# ax[1].tight_layout()
plt.tight_layout()
# plt.legend()

plt.show()